<a href="https://colab.research.google.com/github/dhanushpachabhatla/NLP/blob/main/word2vec_for_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import pandas as pd

In [5]:
# Read using engine='python' and save to a clean CSV
df = pd.read_csv('IMDB Dataset.csv', engine='python', on_bad_lines='skip')
df.to_csv('IMDB_Dataset_clean.csv', index=False)

In [6]:
df = df.iloc[:10000]

In [7]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [8]:
df.drop_duplicates(inplace=True)

In [9]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [10]:
df['review'] = df['review'].apply(remove_tags)

In [11]:
df['review'] = df['review'].apply(lambda x:x.lower())

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [14]:
df['review']

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [15]:
!pip install gensim

In [16]:
import gensim

In [20]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [17]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [21]:
story = []
for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))


In [22]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [23]:
model.build_vocab(story)

In [24]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5850545, 6186875)

In [25]:
len(model.wv.index_to_key)

31845

In [26]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [27]:
document_vector(df['review'].values[0])

array([-0.04152966,  0.40066552,  0.2272707 , -0.03860345, -0.01927529,
       -0.50978416,  0.13453211,  0.8348189 , -0.2240644 , -0.14375933,
       -0.2350125 , -0.5413904 , -0.02810611,  0.25996378,  0.22926997,
       -0.22832994,  0.11152114, -0.5434717 , -0.07136171, -0.7360444 ,
        0.07797845,  0.09734044,  0.22790511, -0.2624658 , -0.23329854,
       -0.18477233, -0.2354096 , -0.24509472, -0.43993402, -0.03865805,
        0.3944676 ,  0.08972777,  0.02627932, -0.17350253, -0.3695076 ,
        0.3702641 ,  0.03929527, -0.2488907 , -0.17762627, -0.750629  ,
        0.06693354, -0.26424268, -0.02825491,  0.02361896,  0.4348748 ,
       -0.19389533, -0.3926365 , -0.20244266,  0.14798345,  0.31403407,
        0.16177945, -0.27259338, -0.25937623, -0.18584289, -0.38880134,
        0.04547076,  0.37484148,  0.22327694, -0.30562982,  0.06509357,
       -0.07465009,  0.18476284, -0.20369056, -0.01814944, -0.40890515,
        0.4276432 ,  0.00731201,  0.14127137, -0.5583893 ,  0.30

In [28]:
from tqdm import tqdm

In [29]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [06:40<00:00, 24.93it/s]


In [30]:
X = np.array(X)

In [31]:
X[0]

array([-0.04152966,  0.40066552,  0.2272707 , -0.03860345, -0.01927529,
       -0.50978416,  0.13453211,  0.8348189 , -0.2240644 , -0.14375933,
       -0.2350125 , -0.5413904 , -0.02810611,  0.25996378,  0.22926997,
       -0.22832994,  0.11152114, -0.5434717 , -0.07136171, -0.7360444 ,
        0.07797845,  0.09734044,  0.22790511, -0.2624658 , -0.23329854,
       -0.18477233, -0.2354096 , -0.24509472, -0.43993402, -0.03865805,
        0.3944676 ,  0.08972777,  0.02627932, -0.17350253, -0.3695076 ,
        0.3702641 ,  0.03929527, -0.2488907 , -0.17762627, -0.750629  ,
        0.06693354, -0.26424268, -0.02825491,  0.02361896,  0.4348748 ,
       -0.19389533, -0.3926365 , -0.20244266,  0.14798345,  0.31403407,
        0.16177945, -0.27259338, -0.25937623, -0.18584289, -0.38880134,
        0.04547076,  0.37484148,  0.22327694, -0.30562982,  0.06509357,
       -0.07465009,  0.18476284, -0.20369056, -0.01814944, -0.40890515,
        0.4276432 ,  0.00731201,  0.14127137, -0.5583893 ,  0.30

In [32]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [33]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [34]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [36]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7766649974962444